In [1]:
from tkinter import *
from tkinter import ttk,messagebox
import pymysql
from tkinter.messagebox import showinfo,showerror
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import cv2,os
import csv
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time


In [3]:
############################################# IMPORTING ################################################
from tkinter import *
from tkinter import ttk,messagebox
import pymysql
from tkinter.messagebox import showinfo,showerror
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import cv2,os
import csv
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time


########################## Sistem Kehadiran Berbasis Pengenalan Wajah ##########################################
class Face:
    
    def __init__(self,root):
        
        self.window = root
        self.window.geometry("1250x720+10+0")
        self.window.resizable(True,False)
        self.window.title("Attendance System")
        self.window.configure(background='#262523')    
        self.window.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')

        ############################################# FUNCTIONS ################################################

        def assure_path_exists(path):
            dir = os.path.dirname(path)
            if not os.path.exists(dir):
                os.makedirs(dir)

        ##################################################################################

        def tick():
            time_string = time.strftime('%H:%M:%S')
            clock.config(text=time_string)
            clock.after(200,tick)

        ###################################################################################

        def contact():
            mess._show(title='Contact us', message="Please contact us on : 'juanfakhri02@gmail.com' ")

        ###################################################################################

        def check_haarcascadefile():
            exists = os.path.isfile("haarcascade_frontalface_default.xml")
            if exists:
                pass
            else:
                mess._show(title='Some file missing', message='Please contact us for help')
                window.destroy()

        
        ######################################################################################

        def clear():
            txt.delete(0, 'end')
            res = "1)Ambil Gambar  >>>  2)Simpan Profil"
            message1.configure(text=res)


        def clear2():
            txt2.delete(0, 'end')
            res = "1)Ambil Gambar  >>>  2)Simpan Profil"
            message1.configure(text=res)

        #######################################################################################

        def TakeImages():
            check_haarcascadefile()
            columns = ['SERIAL NO.', '', 'NIM', '', 'NAME']
            assure_path_exists("StudentDetails/")
            assure_path_exists("TrainingImage/")
            serial = 0
            exists = os.path.isfile("StudentDetails\StudentDetails.csv")
            if exists:
                with open("StudentDetails\StudentDetails.csv", 'r') as csvFile1:
                    reader1 = csv.reader(csvFile1)
                    for l in reader1:
                        serial = serial + 1
                serial = (serial // 2)
                csvFile1.close()
            else:
                with open("StudentDetails\StudentDetails.csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(columns)
                    serial = 1
                csvFile1.close()
            Id = (txt.get())
            name = (txt2.get())
            if ((name.isalpha()) or (' ' in name)):
                cam = cv2.VideoCapture(0)
                harcascadePath = "haarcascade_frontalface_default.xml"
                detector = cv2.CascadeClassifier(harcascadePath)
                sampleNum = 0
                while (True):
                    ret, img = cam.read()
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    faces = detector.detectMultiScale(gray, 1.3, 5)
                    for (x, y, w, h) in faces:
                        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                        # incrementing sample number
                        sampleNum = sampleNum + 1
                        # saving the captured face in the dataset folder TrainingImage
                        cv2.imwrite("TrainingImage\ " + name + "." + str(serial) + "." + Id + '.' + str(sampleNum) + ".jpg",
                                    gray[y:y + h, x:x + w])
                        # display the frame
                        cv2.imshow('Taking Images', img)
                    # wait for 100 miliseconds
                    if cv2.waitKey(100) & 0xFF == ord('q'):
                        break
                    # break if the sample number is morethan 100
                    elif sampleNum > 100:
                        break
                cam.release()
                cv2.destroyAllWindows()
                res = "Images Taken for ID : " + Id
                row = [serial, '', Id, '', name]
                with open('StudentDetails\StudentDetails.csv', 'a+') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(row)
                csvFile.close()
                message1.configure(text=res)
            else:
                if (name.isalpha() == False):
                    res = "Enter Correct name"
                    message.configure(text=res)

        ########################################################################################

        def TrainImages():
            check_haarcascadefile()
            assure_path_exists("TrainingImageLabel/")
            recognizer = cv2.face_LBPHFaceRecognizer.create()
            harcascadePath = "haarcascade_frontalface_default.xml"
            detector = cv2.CascadeClassifier(harcascadePath)
            faces, ID = getImagesAndLabels("TrainingImage")
            try:
                recognizer.train(faces, np.array(ID))
            except:
                mess._show(title='No Registrations', message='Please Register someone first!!!')
                return
            recognizer.save("TrainingImageLabel\Trainner.yml")
            res = "Profile Saved Successfully"
            message1.configure(text=res)
            message.configure(text='Total Pendaftaran hingga sekarang  : ' + str(ID[0]))

        ############################################################################################3

        def getImagesAndLabels(path):
            # get the path of all the files in the folder
            imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
            # create empth face list
            faces = []
            # create empty ID list
            Ids = []
            # now looping through all the image paths and loading the Ids and the images
            for imagePath in imagePaths:
                # loading the image and converting it to gray scale
                pilImage = Image.open(imagePath).convert('L')
                # Now we are converting the PIL image into numpy array
                imageNp = np.array(pilImage, 'uint8')
                # getting the Id from the image
                ID = int(os.path.split(imagePath)[-1].split(".")[1])
                # extract the face from the training image sample
                faces.append(imageNp)
                Ids.append(ID)
            return faces, Ids

        ###########################################################################################

        def TrackImages():
            check_haarcascadefile()
            assure_path_exists("Attendance/")
            assure_path_exists("StudentDetails/")
            for k in tv.get_children():
                tv.delete(k)
            msg = ''
            i = 0
            j = 0
            recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
            exists3 = os.path.isfile("TrainingImageLabel\Trainner.yml")
            if exists3:
                recognizer.read("TrainingImageLabel\Trainner.yml")
            else:
                mess._show(title='Data Missing', message='Please click on Save Profile to reset data!!')
                return
            harcascadePath = "haarcascade_frontalface_default.xml"
            faceCascade = cv2.CascadeClassifier(harcascadePath);

            cam = cv2.VideoCapture(0)
            font = cv2.FONT_HERSHEY_SIMPLEX
            col_names = ['NIM', '', 'Name', '', 'Date', '', 'Time']
            exists1 = os.path.isfile("StudentDetails\StudentDetails.csv")
            if exists1:
                df = pd.read_csv("StudentDetails\StudentDetails.csv")
            else:
                mess._show(title='Details Missing', message='Students details are missing, please check!')
                cam.release()
                cv2.destroyAllWindows()
                window.destroy()
            while True:
                ret, im = cam.read()
                gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                faces = faceCascade.detectMultiScale(gray, 1.2, 5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(im, (x, y), (x + w, y + h), (225, 0, 0), 2)
                    serial, conf = recognizer.predict(gray[y:y + h, x:x + w])
                    if (conf < 50):
                        ts = time.time()
                        date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
                        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                        aa = df.loc[df['SERIAL NO.'] == serial]['NAME'].values
                        ID = df.loc[df['SERIAL NO.'] == serial]['NIM'].values
                        ID = str(ID)
                        ID = ID[1:-1]
                        bb = str(aa)
                        bb = bb[2:-2]
                        attendance = [str(ID), '', bb, '', str(date), '', str(timeStamp)]

                    else:
                        Id = 'Unknown'
                        bb = str(Id)
                    cv2.putText(im, str(bb), (x, y + h), font, 1, (255, 255, 255), 2)
                cv2.imshow('Taking Attendance', im)
                if (cv2.waitKey(1) == ord('q')):
                    break
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
            exists = os.path.isfile("Attendance\Attendance_" + date + ".csv")
            if exists:
                with open("Attendance\Attendance_" + date + ".csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(attendance)
                csvFile1.close()
            else:
                with open("Attendance\Attendance_" + date + ".csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(col_names)
                    writer.writerow(attendance)
                csvFile1.close()
            with open("Attendance\Attendance_" + date + ".csv", 'r') as csvFile1:
                reader1 = csv.reader(csvFile1)
                for lines in reader1:
                    i = i + 1
                    if (i > 1):
                        if (i % 2 != 0):
                            iidd = str(lines[0]) + '   '
                            tv.insert('', 0, text=iidd, values=(str(lines[2]), str(lines[4]), str(lines[6])))
            csvFile1.close()
            cam.release()
            cv2.destroyAllWindows()

        ######################################## USED STUFFS ############################################

        global key
        key = ''

        ts = time.time()
        date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
        day,month,year=date.split("-")

        mont={'01':'January',
              '02':'February',
              '03':'March',
              '04':'April',
              '05':'May',
              '06':'June',
              '07':'July',
              '08':'August',
              '09':'September',
              '10':'October',
              '11':'November',
              '12':'December'
              }

        ######################################## GUI FRONT-END ###########################################
        frame1 = tk.Frame(self.window, bg="#00aeff")
        frame1.place(relx=0.11, rely=0.17, relwidth=0.39, relheight=0.80)

        frame2 = tk.Frame(self.window, bg="#00aeff")
        frame2.place(relx=0.51, rely=0.17, relwidth=0.38, relheight=0.80)

        message3 = tk.Label(self.window, text="Sistem Kehadiran Berbasis Pengenalan Wajah" ,fg="white",bg="#262523" ,width=55 ,height=1,font=('times', 29, ' bold '))
        message3.place(x=10, y=10)

        frame3 = tk.Frame(self.window, bg="#c4c6ce")
        frame3.place(relx=0.52, rely=0.09, relwidth=0.09, relheight=0.07)

        frame4 = tk.Frame(self.window, bg="#c4c6ce")
        frame4.place(relx=0.36, rely=0.09, relwidth=0.16, relheight=0.07)

        datef = tk.Label(frame4, text = day+"-"+mont[month]+"-"+year+"  |  ", fg="orange",bg="#262523" ,width=55 ,height=1,font=('times', 22, ' bold '))
        datef.pack(fill='both',expand=1)

        clock = tk.Label(frame3,fg="orange",bg="#262523" ,width=55 ,height=1,font=('times', 22, ' bold '))
        clock.pack(fill='both',expand=1)
        tick()

        head2 = tk.Label(frame2, text="                       Untuk Pendaftaran Baru                       ", fg="black",bg="#3ece48" ,font=('times', 17, ' bold ') )
        head2.grid(row=0,column=0)

        head1 = tk.Label(frame1, text="                 Untuk Yang Sudah Terdaftar                       ", fg="black",bg="#3ece48" ,font=('times', 17, ' bold ') )
        head1.place(x=0,y=0)

        lbl = tk.Label(frame2, text="Enter Nim",width=20  ,height=1  ,fg="black"  ,bg="#00aeff" ,font=('times', 17, ' bold ') )
        lbl.place(x=80, y=55)

        txt = tk.Entry(frame2,width=32 ,fg="black",font=('times', 15, ' bold '))
        txt.place(x=30, y=88)

        lbl2 = tk.Label(frame2, text="Enter Name",width=20  ,fg="black"  ,bg="#00aeff" ,font=('times', 17, ' bold '))
        lbl2.place(x=80, y=140)

        txt2 = tk.Entry(frame2,width=32 ,fg="black",font=('times', 15, ' bold ')  )
        txt2.place(x=30, y=173)

        message1 = tk.Label(frame2, text="1)Ambil Gambar  >>>  2)Simpan Profil" ,bg="#00aeff" ,fg="black"  ,width=39 ,height=1, activebackground = "yellow" ,font=('times', 15, ' bold '))
        message1.place(x=7, y=230)

        message = tk.Label(frame2, text="" ,bg="#00aeff" ,fg="black"  ,width=39,height=1, activebackground = "yellow" ,font=('times', 16, ' bold '))
        message.place(x=7, y=450)

        lbl3 = tk.Label(frame1, text="Attendance",width=20  ,fg="black"  ,bg="#00aeff"  ,height=1 ,font=('times', 17, ' bold '))
        lbl3.place(x=100, y=115)
        ###################################################################
        res=0
        exists = os.path.isfile("StudentDetails\StudentDetails.csv")
        if exists:
            with open("StudentDetails\StudentDetails.csv", 'r') as csvFile1:
                reader1 = csv.reader(csvFile1)
                for l in reader1:
                    res = res + 1
            res = (res // 2) - 1
            csvFile1.close()
        else:
            res = 0
        message.configure(text='Total Pendaftaran hingga sekarang  : '+str(res))
        
        
        ##################### MENUBAR #################################

        self.menubar = tk.Menu(self.window,relief='ridge')
        self.window.configure(menu=self.menubar)
        filemenu = tk.Menu(self.menubar,tearoff=0)
        filemenu.add_command(label='Contact Us', command = contact)
        filemenu.add_command(label='Exit',command = self.window.destroy)
        self.menubar.add_cascade(label='Help',font=('times', 29, ' bold '),menu=filemenu)

        ################## TREEVIEW ATTENDANCE TABLE ####################

        tv= ttk.Treeview(frame1,height =13,columns = ('name','date','time'))
        tv.column('#0',width=82)
        tv.column('name',width=130)
        tv.column('date',width=133)
        tv.column('time',width=133)
        tv.grid(row=2,column=0,padx=(0,0),pady=(150,0),columnspan=4)
        tv.heading('#0',text ='NIM')
        tv.heading('name',text ='NAME')
        tv.heading('date',text ='DATE')
        tv.heading('time',text ='TIME')

        ###################### SCROLLBAR ################################

        scroll=ttk.Scrollbar(frame1,orient='vertical',command=tv.yview)
        scroll.grid(row=2,column=4,padx=(0,100),pady=(150,0),sticky='ns')
        tv.configure(yscrollcommand=scroll.set)

        ###################### BUTTONS ##################################


        clearButton = tk.Button(frame2, text="Clear", command=clear  ,fg="black"  ,bg="#ea2a2a"  ,width=11 ,activebackground = "white" ,font=('times', 11, ' bold '))
        clearButton.place(x=335, y=86)
        clearButton2 = tk.Button(frame2, text="Clear", command=clear2  ,fg="black"  ,bg="#ea2a2a"  ,width=11 , activebackground = "white" ,font=('times', 11, ' bold '))
        clearButton2.place(x=335, y=172)    
        takeImg = tk.Button(frame2, text="Pindai Wajah", command=TakeImages  ,fg="white"  ,bg="blue"  ,width=34  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        takeImg.place(x=30, y=300)
        trainImg = tk.Button(frame2, text="Simpan Profil", command=TrainImages ,fg="white"  ,bg="blue"  ,width=34  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        trainImg.place(x=30, y=380)
        trackImg = tk.Button(frame1, text="Absen", command=TrackImages  ,fg="black"  ,bg="yellow"  ,width=35  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        trackImg.place(x=30,y=50)
        quitWindow = tk.Button(frame1, text="Quit", command=self.window.destroy  ,fg="black"  ,bg="red"  ,width=35 ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        quitWindow.place(x=30, y=450)

####################################################################################################

############################# GUI LOGIN ##################################################

class Login_window:
    def __init__(self,root):
        self.root2 = root
        self.root2.title("Login Window")
        self.root2.geometry("1290x650+1+0")
        self.root2.configure(background="#021e2f")
        
        #========= Background Image =========
        
        self.btn_img=ImageTk.PhotoImage(file='Template.jpg')
        btn_register = tk.Label(self.root2,image=self.btn_img,bd=0,cursor="hand2").place(x=1,y=0)


        # =============== Frame ================
        login_frame = Frame(self.root2,bg="white",bd=0)
        login_frame.place(x=320,y=117,width=800,height=470)

        self.left=ImageTk.PhotoImage(file='side.png')
        left = Label(self.root2,image=self.left,bg="#021e2f").place(x=90,y=140,width=409,height=415)


        title = Label(login_frame,text="LOGIN HERE",font=("times new roman",30,"bold"),bg="white",fg="#08A3D2").place(x=350,y=50)

        email = Label(login_frame,text="EMAIL ADDRES",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=137)
        self.txt_email = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_email.place(x=257,y=170,width=350,height=35)

        pass_ = Label(login_frame,text="PASSWORD",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=237)
        self.txt_pass_ = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_pass_.place(x=257,y=270,width=350,height=35)

        btn_reg = Button(login_frame,cursor="hand2",text="Register New Account?",font=("times new roman",14),command = self.regis,bg="white",bd=0,fg="#B00857").place(x= 255,y=320)

        btn_login = Button(login_frame,text="Login",font=("times new roman",20,"bold"),cursor="hand2",command = self.login,bg="#B00857",fg="white").place(x= 257,y=370,width=159,height=35)
    def regis(self):
        self.register2 = Toplevel()
        Registration(self.register2)
    
    def login(self):
        if(self.txt_email.get()== "" or self.txt_pass_.get() == ""):
            messagebox.showerror("Error","All Fields Are Required",parent=self.root2)
        else:
            try:
                con = pymysql.connect(host="localhost",user="root",password="",database='employee1')
                cur = con.cursor()
                cur.execute("SELECT * FROM employee where email=%s and password = %s",(self.txt_email.get(),self.txt_pass_.get()))
                row = cur.fetchone()
    
                con.close()
                if row == None:
                    messagebox.showerror("Error","Invalid USERNAME & PASSWORD",parent=self.root2)
                else:
                    messagebox.showinfo("Succesfully","Welcome",parent=self.root2)
                    self.project = Toplevel()
                    self.project.iconbitmap('D:\\Belajar_Python_MasterClass\\Pyroject_Semester_2\\gambar.ico')
                    application= Face(self.project)
            except:
                messagebox.showerror("Error",f"Error due to ",parent=self.root2)


############################################# From Register ################################################

class Registration:
    def __init__(self,root):
        self.root = root
        self.root.title("Registration Window")
        self.root.geometry("1290x650+1+0")
        self.root.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')
        self.root.configure(background="#FFFFF0")
        
        #========= Bg Image =========
        self.Bg=ImageTk.PhotoImage(file='b2.jpg')
        Bg = Label(self.root,image=self.Bg,bg="#FFFFF0").place(x=167,y=0,relwidth=1,relheight=1)
        
        
        #========= Left Image =========
        self.left=ImageTk.PhotoImage(file='side.png')
        left = Label(self.root,image=self.left,bg="#FFFFF0").place(x=70,y=100,width=415,height=500)
        
        #========= Register Frame =======
        frame1 = Frame(self.root,bg="white")
        frame1.place(x=470,y=100,width=700,height=500)
        
        title= Label(frame1,text="REGISTER HERE",font=("times new roman",20,"bold"),bg="white",fg="green").place(x=50, y=30)
        
        # -------------- Row1 ----------
        Id = Label(frame1,text="Id",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=100)
        self.txt_Id = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_Id.place(x=50,y=130,width=250)
        
        name = Label(frame1,text="Name",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=100)
        self.txt_name = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_name.place(x=370,y=130,width=250)
        
        
        
        # ---------------- Row2 -----------------
        contact = Label(frame1,text="Contact No",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=170)
        self.txt_contact = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_contact.place(x=50,y=200,width=250)
        
        email = Label(frame1,text="Email",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=170)
        self.txt_email = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_email.place(x=370,y=200,width=250)
        
        # ---------------- Row3 -----------------
        question = Label(frame1,text="Security Question",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=240)
        
        self.cmb_question = ttk.Combobox(frame1,font=("times new roman",13),state= "readonly",justify=CENTER)
        self.cmb_question['values'] = ("Select","Your NIM","Your Best Friend Name","Your Birth Place")
        self.cmb_question.place(x=50,y=270,width=250)
        self.cmb_question.current(0)
        
        answer = Label(frame1,text="Security Answer",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=240)
        self.txt_answer = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_answer.place(x=370,y=270,width=250)
        
        # ---------------- Row4 -----------------
        password = Label(frame1,text="Password",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=310)
        self.txt_password = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_password.place(x=50,y=340,width=250)
        
        cpassword = Label(frame1,text="Confirm Password",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=310)
        self.txt_cpassword = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_cpassword.place(x=370,y=340,width=250)
        
        
        # ----- Terms -----
        self.var_chk = IntVar()
        chk = Checkbutton(frame1,text="I Agree The Terms & Condition",font=("times new roman",12,"bold"),variable=self.var_chk,onvalue= 1,offvalue=0,bg="white")
        chk.place(x=50,y=375)
        
        self.btn_img=ImageTk.PhotoImage(file='register.png')
        btn_register = Button(frame1,image=self.btn_img,bd=0,cursor="hand2",command= self.register_data).place(x=50,y=420)
        
        self.btn_signIn=ImageTk.PhotoImage(file='sign in.png')
        btn_login = Button(self.root,image=self.btn_signIn,bd=0,command= self.Login_data,cursor="hand2").place(x=200,y=530)
    
    def Login_data(self):
        self.root.destroy()
        
        
    def clear(self):
        self.txt_Id.delete(0,END),
        self.txt_name.delete(0,END),
        self.txt_contact.delete(0,END),
        self.txt_email.delete(0,END), 
        self.cmb_question.current(0), 
        self.txt_answer.delete(0,END),
        self.txt_password.delete(0,END),
        self.txt_cpassword.delete(0,END)
    def register_data(self):
        if self.txt_name.get() == "" or self.txt_Id.get() == "" or self.txt_contact.get()=="" or self.txt_email.get() == "" or self.cmb_question.get() == "Select" or self.txt_answer.get() == "":
            messagebox.showerror("Error","All Fields Are Required",parent=self.root)
        elif self.txt_password.get() != self.txt_cpassword.get():
            messagebox.showerror("Error","Password & Confirm Password should be same",parent=self.root)
        elif self.var_chk.get() == 0:
            messagebox.showerror("Error","Please Agree our terms & condition",parent=self.root)
        else:
            try:
            
                con = pymysql.connect(host="localhost",user="root",password="",database='employee1')
                cur = con.cursor()
                cur.execute("SELECT * FROM employee where email=%s",self.txt_email.get())
                row = cur.fetchone()
                #print(row)
                if row != None:
                    messagebox.showerror("Error","User already Exist, Please try with another email",parent=self.root)
                    self.clear()
                else:
                    cur.execute("INSERT INTO employee (id,name,contact,email,question,answer,password) VALUE(%s,%s,%s,%s,%s,%s,%s)",
                                    (self.txt_Id.get(),
                                     self.txt_name.get(),
                                     self.txt_contact.get(),
                                     self.txt_email.get(), 
                                     self.cmb_question.get(), 
                                     self.txt_answer.get(),
                                     self.txt_password.get()
                                    )
                               )
                    con.commit()
                    con.close
                    messagebox.showinfo("Success","Register Succesfully",parent=self.root)
                    self.clear()
            except Exception as es:
                messagebox.showerror("Error",f"Error due to: {str(es)}",parent=self.root)
            
root=Tk()
root.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')
app = Login_window(root)
root.mainloop()

In [2]:
############################################# IMPORTING ################################################
from tkinter import *
from tkinter import ttk,messagebox
import pymysql
from tkinter.messagebox import showinfo,showerror
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import cv2,os
import csv
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time


########################## Sistem Kehadiran Berbasis Pengenalan Wajah ##########################################
class Face:
    
    def __init__(self,root):
        
        self.window = root
        self.window.geometry("1250x720+10+0")
        self.window.resizable(True,False)
        self.window.title("Attendance System")
        self.window.configure(background='#262523')    
        self.window.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')

        ############################################# FUNCTIONS ################################################

        def assure_path_exists(path):
            dir = os.path.dirname(path)
            if not os.path.exists(dir):
                os.makedirs(dir)

        ##################################################################################

        def tick():
            time_string = time.strftime('%H:%M:%S')
            clock.config(text=time_string)
            clock.after(200,tick)

        ###################################################################################

        def contact():
            mess._show(title='Contact us', message="Please contact us on : 'juanfakhri02@gmail.com' ")

        ###################################################################################

        def check_haarcascadefile():
            exists = os.path.isfile("haarcascade_frontalface_default.xml")
            if exists:
                pass
            else:
                mess._show(title='Some file missing', message='Please contact us for help')
                window.destroy()

        
        ######################################################################################

        def clear():
            txt.delete(0, 'end')
            res = "1)Ambil Gambar  >>>  2)Simpan Profil"
            message1.configure(text=res)


        def clear2():
            txt2.delete(0, 'end')
            res = "1)Ambil Gambar  >>>  2)Simpan Profil"
            message1.configure(text=res)

        #######################################################################################

        def TakeImages():
            check_haarcascadefile()
            columns = ['SERIAL NO.', '', 'NIM', '', 'NAME']
            assure_path_exists("StudentDetails/")
            assure_path_exists("TrainingImage/")
            serial = 0
            exists = os.path.isfile("StudentDetails\StudentDetails.csv")
            if exists:
                with open("StudentDetails\StudentDetails.csv", 'r') as csvFile1:
                    reader1 = csv.reader(csvFile1)
                    for l in reader1:
                        serial = serial + 1
                serial = (serial // 2)
                csvFile1.close()
            else:
                with open("StudentDetails\StudentDetails.csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(columns)
                    serial = 1
                csvFile1.close()
            Id = (txt.get())
            name = (txt2.get())
            if ((name.isalpha()) or (' ' in name)):
                cam = cv2.VideoCapture(0)
                harcascadePath = "haarcascade_frontalface_default.xml"
                detector = cv2.CascadeClassifier(harcascadePath)
                sampleNum = 0
                while (True):
                    ret, img = cam.read()
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    faces = detector.detectMultiScale(gray, 1.3, 5)
                    for (x, y, w, h) in faces:
                        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                        # incrementing sample number
                        sampleNum = sampleNum + 1
                        # saving the captured face in the dataset folder TrainingImage
                        cv2.imwrite("TrainingImage\ " + name + "." + str(serial) + "." + Id + '.' + str(sampleNum) + ".jpg",
                                    gray[y:y + h, x:x + w])
                        # display the frame
                        cv2.imshow('Taking Images', img)
                    # wait for 100 miliseconds
                    if cv2.waitKey(60) & 0xFF == ord('q'):
                        break
                    # break if the sample number is morethan 100
                    elif sampleNum > 60:
                        break
                cam.release()
                cv2.destroyAllWindows()
                res = "Images Taken for ID : " + Id
                row = [serial, '', Id, '', name]
                with open('StudentDetails\StudentDetails.csv', 'a+') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(row)
                csvFile.close()
                message1.configure(text=res)
            else:
                if (name.isalpha() == False):
                    res = "Enter Correct name"
                    message.configure(text=res)

        ########################################################################################

        def TrainImages():
            check_haarcascadefile()
            assure_path_exists("TrainingImageLabel/")
            recognizer = cv2.face_LBPHFaceRecognizer.create()
            harcascadePath = "haarcascade_frontalface_default.xml"
            detector = cv2.CascadeClassifier(harcascadePath)
            faces, ID = getImagesAndLabels("TrainingImage")
            try:
                recognizer.train(faces, np.array(ID))
            except:
                mess._show(title='No Registrations', message='Please Register someone first!!!')
                return
            recognizer.save("TrainingImageLabel\Trainner.yml")
            res = "Profile Saved Successfully"
            message1.configure(text=res)
            message.configure(text='Total Pendaftaran hingga sekarang  : ' + str(ID[0]))

        ############################################################################################3

        def getImagesAndLabels(path):
            # get the path of all the files in the folder
            imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
            # create empth face list
            faces = []
            # create empty ID list
            Ids = []
            # now looping through all the image paths and loading the Ids and the images
            for imagePath in imagePaths:
                # loading the image and converting it to gray scale
                pilImage = Image.open(imagePath).convert('L')
                # Now we are converting the PIL image into numpy array
                imageNp = np.array(pilImage, 'uint8')
                # getting the Id from the image
                ID = int(os.path.split(imagePath)[-1].split(".")[1])
                # extract the face from the training image sample
                faces.append(imageNp)
                Ids.append(ID)
            return faces, Ids

        ###########################################################################################

        def TrackImages():
            check_haarcascadefile()
            assure_path_exists("Attendance/")
            assure_path_exists("StudentDetails/")
            for k in tv.get_children():
                tv.delete(k)
            msg = ''
            i = 0
            j = 0
            recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
            exists3 = os.path.isfile("TrainingImageLabel\Trainner.yml")
            if exists3:
                recognizer.read("TrainingImageLabel\Trainner.yml")
            else:
                mess._show(title='Data Missing', message='Please click on Save Profile to reset data!!')
                return
            harcascadePath = "haarcascade_frontalface_default.xml"
            faceCascade = cv2.CascadeClassifier(harcascadePath);

            cam = cv2.VideoCapture(0)
            font = cv2.FONT_HERSHEY_SIMPLEX
            col_names = ['NIM', '', 'Name', '', 'Date', '', 'Time']
            exists1 = os.path.isfile("StudentDetails\StudentDetails.csv")
            if exists1:
                df = pd.read_csv("StudentDetails\StudentDetails.csv")
            else:
                mess._show(title='Details Missing', message='Students details are missing, please check!')
                cam.release()
                cv2.destroyAllWindows()
                window.destroy()
            while True:
                ret, im = cam.read()
                gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                faces = faceCascade.detectMultiScale(gray, 1.2, 5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(im, (x, y), (x + w, y + h), (225, 0, 0), 2)
                    serial, conf = recognizer.predict(gray[y:y + h, x:x + w])
                    if (conf < 40):
                        ts = time.time()
                        date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
                        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                        aa = df.loc[df['SERIAL NO.'] == serial]['NAME'].values
                        ID = df.loc[df['SERIAL NO.'] == serial]['NIM'].values
                        ID = str(ID)
                        ID = ID[1:-1]
                        bb = str(aa)
                        bb = bb[2:-2]
                        attendance = [str(ID), '', bb, '', str(date), '', str(timeStamp)]

                    else:
                        Id = 'Unknown'
                        bb = str(Id)
                    cv2.putText(im, str(bb), (x, y + h), font, 1, (255, 255, 255), 2)
                cv2.imshow('Taking Attendance', im)
                if (cv2.waitKey(1) == ord('q')):
                    break
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
            exists = os.path.isfile("Attendance\Attendance_" + date + ".csv")
            if exists:
                with open("Attendance\Attendance_" + date + ".csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(attendance)
                csvFile1.close()
            else:
                with open("Attendance\Attendance_" + date + ".csv", 'a+') as csvFile1:
                    writer = csv.writer(csvFile1)
                    writer.writerow(col_names)
                    writer.writerow(attendance)
                csvFile1.close()
            with open("Attendance\Attendance_" + date + ".csv", 'r') as csvFile1:
                reader1 = csv.reader(csvFile1)
                for lines in reader1:
                    i = i + 1
                    if (i > 1):
                        if (i % 2 != 0):
                            iidd = str(lines[0]) + '   '
                            tv.insert('', 0, text=iidd, values=(str(lines[2]), str(lines[4]), str(lines[6])))
            csvFile1.close()
            cam.release()
            cv2.destroyAllWindows()

        ######################################## USED STUFFS ############################################

        global key
        key = ''

        ts = time.time()
        date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
        day,month,year=date.split("-")

        mont={'01':'January',
              '02':'February',
              '03':'March',
              '04':'April',
              '05':'May',
              '06':'June',
              '07':'July',
              '08':'August',
              '09':'September',
              '10':'October',
              '11':'November',
              '12':'December'
              }

        ######################################## GUI FRONT-END ###########################################
        frame1 = tk.Frame(self.window, bg="#00aeff")
        frame1.place(relx=0.11, rely=0.17, relwidth=0.39, relheight=0.80)

        frame2 = tk.Frame(self.window, bg="#00aeff")
        frame2.place(relx=0.51, rely=0.17, relwidth=0.38, relheight=0.80)

        message3 = tk.Label(self.window, text="Sistem Kehadiran Berbasis Pengenalan Wajah" ,fg="white",bg="#262523" ,width=55 ,height=1,font=('times', 29, ' bold '))
        message3.place(x=10, y=10)

        frame3 = tk.Frame(self.window, bg="#c4c6ce")
        frame3.place(relx=0.52, rely=0.09, relwidth=0.09, relheight=0.07)

        frame4 = tk.Frame(self.window, bg="#c4c6ce")
        frame4.place(relx=0.36, rely=0.09, relwidth=0.16, relheight=0.07)

        datef = tk.Label(frame4, text = day+"-"+mont[month]+"-"+year+"  |  ", fg="orange",bg="#262523" ,width=55 ,height=1,font=('times', 22, ' bold '))
        datef.pack(fill='both',expand=1)

        clock = tk.Label(frame3,fg="orange",bg="#262523" ,width=55 ,height=1,font=('times', 22, ' bold '))
        clock.pack(fill='both',expand=1)
        tick()

        head2 = tk.Label(frame2, text="                       Untuk Pendaftaran Baru                       ", fg="black",bg="#3ece48" ,font=('times', 17, ' bold ') )
        head2.grid(row=0,column=0)

        head1 = tk.Label(frame1, text="                 Untuk Yang Sudah Terdaftar                       ", fg="black",bg="#3ece48" ,font=('times', 17, ' bold ') )
        head1.place(x=0,y=0)

        lbl = tk.Label(frame2, text="Enter Nim",width=20  ,height=1  ,fg="black"  ,bg="#00aeff" ,font=('times', 17, ' bold ') )
        lbl.place(x=80, y=55)

        txt = tk.Entry(frame2,width=32 ,fg="black",font=('times', 15, ' bold '))
        txt.place(x=30, y=88)

        lbl2 = tk.Label(frame2, text="Enter Name",width=20  ,fg="black"  ,bg="#00aeff" ,font=('times', 17, ' bold '))
        lbl2.place(x=80, y=140)

        txt2 = tk.Entry(frame2,width=32 ,fg="black",font=('times', 15, ' bold ')  )
        txt2.place(x=30, y=173)

        message1 = tk.Label(frame2, text="1)Ambil Gambar  >>>  2)Simpan Profil" ,bg="#00aeff" ,fg="black"  ,width=39 ,height=1, activebackground = "yellow" ,font=('times', 15, ' bold '))
        message1.place(x=7, y=230)

        message = tk.Label(frame2, text="" ,bg="#00aeff" ,fg="black"  ,width=39,height=1, activebackground = "yellow" ,font=('times', 16, ' bold '))
        message.place(x=7, y=450)

        lbl3 = tk.Label(frame1, text="Attendance",width=20  ,fg="black"  ,bg="#00aeff"  ,height=1 ,font=('times', 17, ' bold '))
        lbl3.place(x=100, y=115)
        ###################################################################
        res=0
        exists = os.path.isfile("StudentDetails\StudentDetails.csv")
        if exists:
            with open("StudentDetails\StudentDetails.csv", 'r') as csvFile1:
                reader1 = csv.reader(csvFile1)
                for l in reader1:
                    res = res + 1
            res = (res // 2) - 1
            csvFile1.close()
        else:
            res = 0
        message.configure(text='Total Pendaftaran hingga sekarang  : '+str(res))
        
        
        ##################### MENUBAR #################################

        self.menubar = tk.Menu(self.window,relief='ridge')
        self.window.configure(menu=self.menubar)
        filemenu = tk.Menu(self.menubar,tearoff=0)
        filemenu.add_command(label='Contact Us', command = contact)
        filemenu.add_command(label='Exit',command = self.window.destroy)
        self.menubar.add_cascade(label='Help',font=('times', 29, ' bold '),menu=filemenu)

        ################## TREEVIEW ATTENDANCE TABLE ####################

        tv= ttk.Treeview(frame1,height =13,columns = ('name','date','time'))
        tv.column('#0',width=82)
        tv.column('name',width=130)
        tv.column('date',width=133)
        tv.column('time',width=133)
        tv.grid(row=2,column=0,padx=(0,0),pady=(150,0),columnspan=4)
        tv.heading('#0',text ='NIM')
        tv.heading('name',text ='NAME')
        tv.heading('date',text ='DATE')
        tv.heading('time',text ='TIME')

        ###################### SCROLLBAR ################################

        scroll=ttk.Scrollbar(frame1,orient='vertical',command=tv.yview)
        scroll.grid(row=2,column=4,padx=(0,100),pady=(150,0),sticky='ns')
        tv.configure(yscrollcommand=scroll.set)

        ###################### BUTTONS ##################################


        clearButton = tk.Button(frame2, text="Clear", command=clear  ,fg="black"  ,bg="#ea2a2a"  ,width=11 ,activebackground = "white" ,font=('times', 11, ' bold '))
        clearButton.place(x=335, y=86)
        clearButton2 = tk.Button(frame2, text="Clear", command=clear2  ,fg="black"  ,bg="#ea2a2a"  ,width=11 , activebackground = "white" ,font=('times', 11, ' bold '))
        clearButton2.place(x=335, y=172)    
        takeImg = tk.Button(frame2, text="Pindai Wajah", command=TakeImages  ,fg="white"  ,bg="blue"  ,width=34  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        takeImg.place(x=30, y=300)
        trainImg = tk.Button(frame2, text="Simpan Profil", command=TrainImages ,fg="white"  ,bg="blue"  ,width=34  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        trainImg.place(x=30, y=380)
        trackImg = tk.Button(frame1, text="Absen", command=TrackImages  ,fg="black"  ,bg="yellow"  ,width=35  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        trackImg.place(x=30,y=50)
        quitWindow = tk.Button(frame1, text="Quit", command=self.window.destroy  ,fg="black"  ,bg="red"  ,width=35 ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
        quitWindow.place(x=30, y=450)

####################################################################################################


############################################# IMPORTING ################################################
from tkinter import *
from tkinter import ttk,messagebox
import pymysql
from tkinter.messagebox import showinfo,showerror
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time


############################# GUI LOGIN ##################################################

class Login_window:
    def __init__(self,root):
        self.root2 = root
        self.root2.title("Login Window")
        self.root2.geometry("1290x650+1+0")
        self.root2.configure(background="#021e2f")
        
        #========= Background Image =========
        
        self.btn_img=ImageTk.PhotoImage(file='Template.jpg')
        btn_register = tk.Label(self.root2,image=self.btn_img,bd=0,cursor="hand2").place(x=1,y=0)


        # =============== Frame ================
        login_frame = Frame(self.root2,bg="white",bd=0)
        login_frame.place(x=320,y=117,width=800,height=470)

        self.left=ImageTk.PhotoImage(file='side.png')
        left = Label(self.root2,image=self.left,bg="#021e2f").place(x=90,y=140,width=409,height=415)


        title = Label(login_frame,text="LOGIN HERE",font=("times new roman",30,"bold"),bg="white",fg="#08A3D2").place(x=350,y=50)

        email = Label(login_frame,text="EMAIL ADDRES",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=137)
        self.txt_email = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_email.place(x=257,y=170,width=350,height=35)

        pass_ = Label(login_frame,text="PASSWORD",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=237)
        self.txt_pass_ = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_pass_.place(x=257,y=270,width=350,height=35)

        btn_reg = Button(login_frame,cursor="hand2",text="Register New Account?",font=("times new roman",14),command = self.regis,bg="white",bd=0,fg="#B00857").place(x= 255,y=320)

        btn_login = Button(login_frame,text="Login",font=("times new roman",20,"bold"),cursor="hand2",command = self.login,bg="#B00857",fg="white").place(x= 257,y=370,width=159,height=35)
    def regis(self):
        self.register2 = Toplevel()
        Registration(self.register2)
    
    def login(self):
        if(self.txt_email.get()== "" or self.txt_pass_.get() == ""):
            messagebox.showerror("Error","All Fields Are Required",parent=self.root2)
        else:
            try:
                con = pymysql.connect(host="localhost",user="root",password="",database='employee1')
                cur = con.cursor()
                cur.execute("SELECT * FROM employee where email=%s and password = %s",(self.txt_email.get(),self.txt_pass_.get()))
                row = cur.fetchone()
    
                con.close()
                if row == None:
                    messagebox.showerror("Error","Invalid USERNAME & PASSWORD",parent=self.root2)
                else:
                    messagebox.showinfo("Succesfully","Welcome",parent=self.root2)
                    self.project = Toplevel()
                    self.project.iconbitmap('D:\\Belajar_Python_MasterClass\\Pyroject_Semester_2\\gambar.ico')
                    application= Face(self.project)
            except:
                messagebox.showerror("Error",f"Error due to ",parent=self.root2)


############################################# From Register ################################################

class Registration:
    def __init__(self,root):
        self.root = root
        self.root.title("Registration Window")
        self.root.geometry("1290x650+1+0")
        self.root.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')
        self.root.configure(background="#FFFFF0")
        
        #========= Bg Image =========
        self.Bg=ImageTk.PhotoImage(file='b2.jpg')
        Bg = Label(self.root,image=self.Bg,bg="#FFFFF0").place(x=167,y=0,relwidth=1,relheight=1)
        
        
        #========= Left Image =========
        self.left=ImageTk.PhotoImage(file='side.png')
        left = Label(self.root,image=self.left,bg="#FFFFF0").place(x=70,y=100,width=415,height=500)
        
        #========= Register Frame =======
        frame1 = Frame(self.root,bg="white")
        frame1.place(x=470,y=100,width=700,height=500)
        
        title= Label(frame1,text="REGISTER HERE",font=("times new roman",20,"bold"),bg="white",fg="green").place(x=50, y=30)
        
        # -------------- Row1 ----------
        Id = Label(frame1,text="Id",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=100)
        self.txt_Id = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_Id.place(x=50,y=130,width=250)
        
        name = Label(frame1,text="Name",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=100)
        self.txt_name = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_name.place(x=370,y=130,width=250)
        
        
        
        # ---------------- Row2 -----------------
        contact = Label(frame1,text="Contact No",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=170)
        self.txt_contact = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_contact.place(x=50,y=200,width=250)
        
        email = Label(frame1,text="Email",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=170)
        self.txt_email = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_email.place(x=370,y=200,width=250)
        
        # ---------------- Row3 -----------------
        question = Label(frame1,text="Security Question",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=240)
        
        self.cmb_question = ttk.Combobox(frame1,font=("times new roman",13),state= "readonly",justify=CENTER)
        self.cmb_question['values'] = ("Select","Your NIM","Your Best Friend Name","Your Birth Place")
        self.cmb_question.place(x=50,y=270,width=250)
        self.cmb_question.current(0)
        
        answer = Label(frame1,text="Security Answer",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=240)
        self.txt_answer = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_answer.place(x=370,y=270,width=250)
        
        # ---------------- Row4 -----------------
        password = Label(frame1,text="Password",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=50, y=310)
        self.txt_password = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_password.place(x=50,y=340,width=250)
        
        cpassword = Label(frame1,text="Confirm Password",font=("times new roman",15,"bold"),bg="white",fg="gray").place(x=370, y=310)
        self.txt_cpassword = Entry(frame1,font=("times new roman",15),bg="lightgray")
        self.txt_cpassword.place(x=370,y=340,width=250)
        
        
        # ----- Terms -----
        self.var_chk = IntVar()
        chk = Checkbutton(frame1,text="I Agree The Terms & Condition",font=("times new roman",12,"bold"),variable=self.var_chk,onvalue= 1,offvalue=0,bg="white")
        chk.place(x=50,y=375)
        
        self.btn_img=ImageTk.PhotoImage(file='register.png')
        btn_register = Button(frame1,image=self.btn_img,bd=0,cursor="hand2",command= self.register_data).place(x=50,y=420)
        
        self.btn_signIn=ImageTk.PhotoImage(file='sign in.png')
        btn_login = Button(self.root,image=self.btn_signIn,bd=0,command= self.Login_data,cursor="hand2").place(x=200,y=530)
    
    def Login_data(self):
        self.root.destroy()
        
        
    def clear(self):
        self.txt_Id.delete(0,END),
        self.txt_name.delete(0,END),
        self.txt_contact.delete(0,END),
        self.txt_email.delete(0,END), 
        self.cmb_question.current(0), 
        self.txt_answer.delete(0,END),
        self.txt_password.delete(0,END),
        self.txt_cpassword.delete(0,END)
    def register_data(self):
        if self.txt_name.get() == "" or self.txt_Id.get() == "" or self.txt_contact.get()=="" or self.txt_email.get() == "" or self.cmb_question.get() == "Select" or self.txt_answer.get() == "":
            messagebox.showerror("Error","All Fields Are Required",parent=self.root)
        elif self.txt_password.get() != self.txt_cpassword.get():
            messagebox.showerror("Error","Password & Confirm Password should be same",parent=self.root)
        elif self.var_chk.get() == 0:
            messagebox.showerror("Error","Please Agree our terms & condition",parent=self.root)
        else:
            try:
            
                con = pymysql.connect(host="localhost",user="root",password="",database='employee1')
                cur = con.cursor()
                cur.execute("SELECT * FROM employee where email=%s",self.txt_email.get())
                row = cur.fetchone()
                #print(row)
                if row != None:
                    messagebox.showerror("Error","User already Exist, Please try with another email",parent=self.root)
                    self.clear()
                else:
                    cur.execute("INSERT INTO employee (id,name,contact,email,question,answer,password) VALUE(%s,%s,%s,%s,%s,%s,%s)",
                                    (self.txt_Id.get(),
                                     self.txt_name.get(),
                                     self.txt_contact.get(),
                                     self.txt_email.get(), 
                                     self.cmb_question.get(), 
                                     self.txt_answer.get(),
                                     self.txt_password.get()
                                    )
                               )
                    con.commit()
                    con.close
                    messagebox.showinfo("Success","Register Succesfully",parent=self.root)
                    self.clear()
            except Exception as es:
                messagebox.showerror("Error",f"Error due to: {str(es)}",parent=self.root)
            
root=Tk()
root.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')
app = Login_window(root)
root.mainloop()

In [3]:
class Login_window():
    def __init__(self,root):
        self.root2 = root
        self.root2.title("Login Window")
        self.root2.geometry("1290x650+1+0")
        self.root2.configure(background="#021e2f")

        #========= Background Image =========

        left_lbl = Label(self.root2,bg="#08A3D2",bd=0)
        left_lbl.place(x=0,y=0,relheight=1,width=600)

        rigth_lbl = Label(self.root2,bg="#031F3C",bd=0)
        rigth_lbl.place(x=500,y=0,relwidth=1,relheight=1)


        # =============== Frame ================
        login_frame = Frame(self.root2,bg="white",bd=0)
        login_frame.place(x=320,y=100,width=810,height=500)

        self.left=ImageTk.PhotoImage(file='side.png')
        left = Label(self.root2,image=self.left,bg="#021e2f").place(x=90,y=120,width=413,height=460)


        title = Label(login_frame,text="LOGIN HERE",font=("times new roman",30,"bold"),bg="white",fg="#08A3D2").place(x=350,y=50)

        email = Label(login_frame,text="EMAIL ADDRES",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=137)
        self.txt_email = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_email.place(x=257,y=170,width=350,height=35)

        pass_ = Label(login_frame,text="PASSWORD",font=("times new roman",18,"bold"),bg="white",fg="gray").place(x=257,y=237)
        self.txt_pass_ = Entry(login_frame,font=("times new roman",15),bg="lightgray")
        self.txt_pass_.place(x=257,y=270,width=350,height=35)

        btn_reg = Button(login_frame,cursor="hand2",text="Register New Account?",font=("times new roman",14),bg="white",bd=0,fg="#B00857").place(x= 255,y=320)

        btn_login = Button(login_frame,text="Login",font=("times new roman",20,"bold"),cursor="hand2",command = self.login,bg="#B00857",fg="white").place(x= 257,y=370,width=159,height=35)
    
    def clear(self):
        self.txt_email.delete(0,END),
        self.txt_pass_.delete(0,END)
    
    def login(self):
        if(self.txt_email.get()== "" or self.txt_pass_.get() == ""):
            messagebox.showerror("Error","All Fields Are Required",parent=self.root2)
        else:
            try:
                con = pymysql.connect(host="localhost",user="root",password="",database='employee1')
                cur = con.cursor()
                cur.execute("SELECT * FROM employee where email=%s and password = %s",(self.txt_email.get(),self.txt_pass_.get()))
                row = cur.fetchone()
                if row == None:
                    messagebox.showerror("Error","Invalid USERNAME & PASSWORD",parent=self.root2)
                else:
                    messagebox.showinfo("Succesfully","Welcome",parent=self.root2)
            except Exception as es:
                messagebox.showerror("Error",f"Error due to {str(es)}",parent=self.root2)
                
        
root=Tk()
root.iconbitmap('D:\\Belajar_Python_MasterClass\\Face Recognition\\gambar.ico')
app = Login_window(root)
root.mainloop()

In [ ]:
a.

In [ ]:
userID = 'urEmailDomain'
home = 'urHomeWindow'
def login():
    global userID
    try:
        connection = pymysql.connect(host="localhost",user="root",password="",database='test2')
    except:
        print("You are not connected to server(localhost)")
    else:
        print("connected Succesfully")
        print("Enter ur Email and password")
        Email = User.get()
        Password = Pass.get()
        cur = connection.cursor()
        query = "SELECT email,password FROM users"
        cur.execute(query)
        for (email,pas) in cur:
            if Email == email and Password == pas:
                login == True
                break
            else:
                login == False
        userID = (Email.split('@')[0])
        if login == True:
            print('Logged in Succesfully as',userID)
            newWindow()
        elif login == False:
            print('Email or password is wrong')
        cur.close()
        connection.close()
def 

In [5]:
class Registration():
    def form():
        try:
            connection = pymysql.connect(host="localhost",user="root",password="",database='test2')
        except:
            print("You are not connected to server(localhost)")
        else:
            cur = connection.cursor()
            print("connected Succesfully! feel the from")
#===================form Design==============================#
        rFrom = TK()
        rFrom.title("New User Registration")
        rFrom.geometry('300X500')
        rFrom.config(bg='powder blue')
        formName = Label(rFrom,text= 'New User Registration',fg='blue',font=('arial',20,'bold')).pack()
        
#       Get user full name ============
        L1 = Label(rFrom,='Enter you Name')
        L1.place(x=30, y=100)
        regUser = Entry(rFrom)
        regUser.place(x=140, y=100)
#       Get user Email id =============
        L2 = Label(rFrom,='Email')
        L2.place(x=30, y=130)
        regEmail = Entry(rFrom)
        regEmail.place(x=140, y=130)